### Notebook for scrapping Postal_Code , Borough and Neighborhood from given Wikipedia page.

#### Import Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# get response from the given wikipedia page

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
# convert above response in HTML format using BeautifulSoup.

soup = BeautifulSoup(website_url,'lxml')

In [4]:
# In the above HTML page table of interest is "wikitable sortable".
# We can access the table using below code.
my_table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
columns = []           # list for header / columns of the table.
all_data = []          # list for all <td> values. 
postal_codes = []      # list for all postal codes i.e. every third item of all_data[] list starting from 0 index.
borough = []           # list for all borough i.e. every third item of all_data[] list starting from 1 index.
neighborhood = []      # list for all neighnorhood i.e. every third item of all_data[] list starting from 2 index.

In [6]:
# get all <th> values from the table i.e. column names for our dataframe.
# removing 'newline' from last column.

cols = my_table.find_all('th')
for c in cols:
    columns.append(c.text)
columns[2] = columns[2].replace('\n','')
columns

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
#populate all_data[] with all <td> values and removing 'newline' from last column.

all_d = my_table.find_all('td')
for a in all_d:
    all_data.append(a.text)
    
for i in range(len(all_data)):
    all_data[i] = all_data[i].replace('\n','')


In [8]:
#polulate Postal_Codes[] , Borough[] and Neighborhood[].

postal_codes = all_data[::3]
borough = all_data[1::3]
neighborhood = all_data[2::3]


In [10]:
# Creating Dataframe from above data.
df = pd.DataFrame() #creating empty dataframe.

#Populating columns
df[columns[0]] = postal_codes
df[columns[1]] = borough
df[columns[2]] = neighborhood

### Now we need to clean the above dataframe using below conditions:
1.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

2.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park

In [11]:
df = df[df.Borough != 'Not assigned'] #satisfies first condition.
df.Neighbourhood.replace('Not assigned' , df.Borough , inplace = True)  #satisfies second condition.
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [12]:
# shape of the resulting dataframe
df.shape

(211, 3)

In [13]:
#import geospatial file into dataframe
df_geo = pd.read_csv(r'C:\Users\aksha\Desktop\projects\Coursera_Capstone\Geospatial_Coordinates.csv')

In [14]:
# lets compare columns of both dataframe in order to join and get latitude and longitudes

print(df.columns)
print(df_geo.columns)

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')
Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


In [15]:
#change column name of first dataframe from 'Postcode' to 'Postal Code'

df.rename(columns={'Postcode' : 'Postal Code'}, inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [16]:
# Now lets merge both dataframes on 'Postal code'

df = df.merge(df_geo , on='Postal Code')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


## End of Content